# Stability estimates

Imports

In [ ]:
import sys
import numpy as np
import tabulate
import time

import logging
import copy

import matplotlib.pyplot as plt
import os
import sys
import re
import glob
import ast
import os.path as op
import pickle
import scipy
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import scipy as sp

from collections import defaultdict

from copy import deepcopy
import copy

from sklearn.pipeline import Pipeline
from sklearn.utils import resample
from sklearn.covariance import GraphicalLassoCV
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LinearRegression

from itertools import combinations
from scipy.stats import ttest_ind

from scipy.stats import pearsonr
from scipy.spatial import distance
import scipy.stats as stats

import networkx as nx
import glob, os
from functools import partial

from scipy.stats import pearsonr

import pickle

import warnings
warnings.filterwarnings("ignore")

Constatnts

In [ ]:
mapping = {
    "RRQ": "1Q",
    "DASS-21 Stress": "2Q",
    "DASS-21 Dep": "3Q",
    "STAI-T": "4Q",
    "STAI-S Diff": '5Q',
    "BIS": "6Q",
    "OBSESS": "7Q",
    "HOARD": "8Q",
    "ORD": "9Q",
    "CHECK": "10Q",
    "WBSI": "11Q",
    "IUS-P": "12Q",
    "IUS-I": "13Q",
    "SES": "14Q",
    'BAS_D': "15Q",
    'BAS_PRZY': "16Q",
    'BAS_NAG': "17Q",
    'INDEC_F': "18Q",
    'PUN': "19Q",
    'REW': "20Q",
    'HARM': "21Q",
    'T-CTR': "22Q",
    "OT": "23Q",
    'OB_PERF': "24Q",
    'PS': "25Q",
    'G_SE': "26Q",
    'AMB': "27Q",
    'PRED': "28Q",
    'STAND': "29Q",   
    "Age": "1C",
    "Handness": "2C",
    'e_ERN': "1E",
    'e_LT_F': "4C",
    'performance': "3C",
    'e_CRN': "2E",
    'e_LT_F2_C': "4C",
    'performance': "3C",
}

ern_order = ['1E', '1Q', '2Q', '3Q', '4Q', '5Q', '6Q', '7Q', '8Q', '9Q', '10Q', '11Q', '12Q', '13Q', '14Q', '15Q', '16Q', '17Q', '18Q', '19Q', '20Q', '21Q', '22Q', '23Q', '24Q', '25Q', '26Q', '27Q', '28Q', '29Q',]
ern_cov_order = ['1E', '1Q', '2Q', '3Q', '4Q', '5Q', '6Q', '7Q', '8Q', '9Q', '10Q', '11Q', '12Q', '13Q', '14Q', '15Q', '16Q', '17Q', '18Q', '19Q', '20Q', '21Q', '22Q', '23Q', '24Q', '25Q', '26Q', '27Q', '28Q', '29Q', '1C', '2C', '3C', '4C']
crn_order = ['2E', '1Q', '2Q', '3Q', '4Q', '5Q', '6Q', '7Q', '8Q', '9Q', '10Q', '11Q', '12Q', '13Q', '14Q', '15Q', '16Q', '17Q', '18Q', '19Q', '20Q', '21Q', '22Q', '23Q', '24Q', '25Q', '26Q', '27Q', '28Q', '29Q',]
crn_cov_order = ['2E', '1Q', '2Q', '3Q', '4Q', '5Q', '6Q', '7Q', '8Q', '9Q', '10Q', '11Q', '12Q', '13Q', '14Q', '15Q', '16Q', '17Q', '18Q', '19Q', '20Q', '21Q', '22Q', '23Q', '24Q', '25Q', '26Q', '27Q', '28Q', '29Q', '1C', '2C', '3C', '4C']

orders = [ern_order, ern_cov_order, crn_order, crn_cov_order]

## Load data

In [ ]:
dataset = 'train'
test = False if dataset == 'train' else False

Loading training data

In [ ]:
results_ern_df = pd.read_pickle(f"../data/models_pickles_new_dass/ern_models_{dataset}.pkl")
results_ern_lat_demo_df = pd.read_pickle(f"../data/models_pickles_new_dass/ern_cov_fal_models_{dataset}.pkl")
results_crn_df = pd.read_pickle(f"../data/models_pickles_new_dass/crn_models_{dataset}.pkl")
results_crn_lat_demo_df = pd.read_pickle(f"../data/models_pickles_new_dass/crn_cov_fal2_models_{dataset}.pkl")

Loading testing data

In [ ]:
results_ern_test_df = pd.read_pickle(f"../data/models_pickles_new_dass/ern_models_test.pkl")
results_ern_lat_demo_test_df = pd.read_pickle(f"../data/models_pickles_new_dass/ern_cov_fal_models_test.pkl")
results_crn_test_df = pd.read_pickle(f"../data/models_pickles_new_dass/crn_models_test.pkl")
results_crn_lat_demo_test_df = pd.read_pickle(f"../data/models_pickles_new_dass/crn_cov_fal2_models_test.pkl")

# fill nan
results_crn_lat_demo_test_df['e_LT_F2_C'] = results_crn_lat_demo_test_df['e_LT_F2_C'].fillna(results_crn_lat_demo_test_df['e_LT_F2_C'].mean())

Prepare datasets

In [ ]:
columns_to_drop_covs = ['WASH', 'NEU', 'Sex']
columns_to_drop = ['WASH', 'NEU']

# train dataset
ern_data_df_ = results_ern_df.drop(columns=columns_to_drop)
ern_cov_fal_data_df_ = results_ern_lat_demo_df.drop(columns=columns_to_drop_covs)
crn_data_df_ = results_crn_df.drop(columns=columns_to_drop)
crn_cov_fal2_data_df_ = results_crn_lat_demo_df.drop(columns=columns_to_drop_covs)

# test dataset
ern_data_df_test_ = results_ern_test_df.drop(columns=columns_to_drop)
ern_cov_fal_data_df_test_ = results_ern_lat_demo_test_df.drop(columns=columns_to_drop_covs)
crn_data_df_test_ = results_crn_test_df.drop(columns=columns_to_drop)
crn_cov_fal2_data_df_test_ = results_crn_lat_demo_test_df.drop(columns=columns_to_drop_covs)

datasets_train = [ern_data_df_, ern_cov_fal_data_df_, crn_data_df_, crn_cov_fal2_data_df_]
datasets_test = [ern_data_df_test_, ern_cov_fal_data_df_test_, crn_data_df_test_, crn_cov_fal2_data_df_test_]

### Read estimated graphs

In [ ]:
dir_ = f"../data/graphs/{dataset}/"

graphs = []

for file in sorted(glob.glob(dir_ + "*.gml")):
    print(file)
    graph = nx.read_gml(file)
    graphs.append(graph)
    
graphs

## 1. Stability of network structure as proportion matrices from stability selection procedure

In [ ]:
def plot_proportion_matrices(
    proportion_matrices, 
    mapping=None, 
    save=False,
):
    proportion_matrix_1 = proportion_matrices[0][0]
    order_1 = proportion_matrices[0][1]
    
    ern_df = proportion_matrix_1.rename(columns=mapping, index=mapping)
    ern_df = ern_df.reindex(columns=order_1, index=order_1)
    ern_mask = np.triu(np.ones_like(ern_df, dtype=bool))

    proportion_matrix_2 = proportion_matrices[1][0]
    order_2 = proportion_matrices[1][1]
    
    crn_df = proportion_matrix_2.rename(columns=mapping, index=mapping)
    crn_df = crn_df.reindex(columns=order_2, index=order_2)
    crn_mask = np.tril(np.ones_like(crn_df, dtype=bool))
    

    # define plotting parameters
    cmap = sns.diverging_palette(100, 7, s=75, l=40, n=5, center="light", as_cmap=True)
    cm = 1/2.54
    dpi = 500
    
    sns.set_style("white")
    plt.rcParams['figure.dpi'] = dpi
    plt.rcParams['figure.figsize'] = [15*cm,15*cm]
    plt.rcParams["font.size"] = 4
    plt.rcParams['ytick.labelsize'] = 6
    plt.rcParams['xtick.labelsize'] = 6
    plt.rcParams['axes.labelsize'] = 7
    plt.rcParams["axes.edgecolor"] = ".15"
    plt.rcParams["axes.linewidth"]  = 0.5

    # create canvas
    figure, axis = plt.subplots(1, 1)

    # draw first proportion matrix
    g_1 = sns.heatmap(
        data = ern_df,
        annot=True,
        mask = ern_mask,
        center=0.5,
        fmt='.2f',
        square=True,
        cmap=cmap,
        cbar=False,
        linewidths=1,
        ax=axis
    )

    # draw second proportion matrix
    g_2 = sns.heatmap(
        data = crn_df,
        annot=True,
        mask = crn_mask,
        center=0.5,
        fmt='.2f',
        square=True,
        cmap=cmap,
        cbar=False,
        linewidths=1,
        ax=axis
    )

    if save != False:
        figure.savefig(f"data/support_proportion/train/{save}.png",  bbox_inches='tight')
    
    
    plt.show()
    return figure

Read proportion matrices from the stability selection procedure

In [ ]:
dir_ = f"../data/support_proportion/train/"

proportion_matrices = []

for file in sorted(glob.glob(dir_ + "proportion_*.pkl")):
    print(file)
    proportion_matrix = pd.read_pickle(file)
    proportion_matrices.append(proportion_matrix)

#### Draw proportion matrices for ERN and CRN models

In [ ]:
proportion_matrices_to_plot = [
    (proportion_matrices[0], ern_order),
    (proportion_matrices[2], crn_order), 
]

fig = plot_proportion_matrices(
    proportion_matrices = proportion_matrices_to_plot, 
    mapping=mapping, 
    save=False,
)

#### Draw proportion matrices for ERN with covariates and CRN with covariates models

In [ ]:
proportion_matrices_to_plot = [
    (proportion_matrices[1], ern_cov_order),
    (proportion_matrices[3], crn_cov_order), 
]

fig = plot_proportion_matrices(
    proportion_matrices = proportion_matrices_to_plot, 
    mapping=mapping, 
    save=False,
)

## 2. Edge-Weight stability using bootstrapping on adaptive average procedure

In [ ]:
def read_from_pickle(path):
    with open(path, 'rb') as file:
        precision_matrices = pickle.load(file)
    return precision_matrices

In [ ]:
def reorder_matrix(matrix, columns, index, mapping, mapping_order, to_numpy=True):
    df = pd.DataFrame(matrix, columns=columns, index=index)
    df = df.rename(columns=mapping, index=mapping)
    df = df.reindex(columns=mapping_order, index=mapping_order)

    result = df.to_numpy() if to_numpy else df
    return result

In [ ]:
def get_edge_weights_stability_estimates(
    precision_matrices, 
    df, 
    mapping, 
    order, 
):
    precision_matrices_list = [reorder_matrix(matrix, df.columns, df.columns, mapping=mapping, mapping_order=order) for matrix in precision_matrices]
    
    # Step 1: Calculate the average matrix
    average_matrix = np.around(sum(precision_matrices_list) / len(precision_matrices_list), decimals=2)

    # Step 2: Calculate the standard error matrix
    std_error_matrix = np.std(precision_matrices_list, axis=0) / np.sqrt(len(precision_matrices_list))

    # Step 3: Calculate the 95% confidence interval values
    z_score = stats.norm.ppf(0.975)  # Z-score for a 95% confidence interval

    n = average_matrix.shape[0]
    upper_triangle = np.triu(np.ones((n, n), dtype=bool), k=1)
    lower_triangle = np.tril(np.ones((n, n), dtype=bool), k=0)

    confidence_interval_low = average_matrix.copy()
    confidence_interval_low[upper_triangle] -= z_score * std_error_matrix[upper_triangle]

    confidence_interval_high = average_matrix.copy()
    confidence_interval_high[upper_triangle] += z_score * std_error_matrix[upper_triangle]

    # Step 4: Combine the confidence interval ranges into the final_matrix as strings
    final_matrix = np.empty_like(average_matrix, dtype=object)
    final_matrix[upper_triangle] = [
    f"[{low:.2f},\n {high:.2f}]" for low, high in zip(confidence_interval_low[upper_triangle], confidence_interval_high[upper_triangle])
    ]
    final_matrix[lower_triangle] = average_matrix[lower_triangle]
    
    df = df.rename(columns=mapping, index=mapping)
    df = df.reindex(columns=order, index=order)
    
    df_ = pd.DataFrame(final_matrix, columns = df.columns, index = df.columns)
    
    return df_

Read bootstrap results

In [ ]:
dir_ = f"../data/bootstrap_results/"

precision_bootstrap_matrices = []

for file in sorted(glob.glob(dir_ + "*.pkl")):
    print(file)
    precision_bootstrap_matrix = read_from_pickle(file)
    precision_bootstrap_matrices.append(precision_bootstrap_matrix)

Estimate edge-weight stability

In [ ]:
stability_estimates = []
for precisions, dataset, order in zip(precision_bootstrap_matrices, datasets_train, orders):
    edge_weights_stability_estimates_df = get_edge_weights_stability_estimates(precisions, dataset, mapping, order)
    stability_estimates.append(edge_weights_stability_estimates_df)

Save results

In [ ]:
for i, df in enumerate(stability_estimates):
    df.to_pickle(f"../data/bootstrap_results/edge_weight_stability/edge_weights_stability_train_2-decim_graph_{i}.pkl")
    df.to_csv(f"../data/bootstrap_results/edge_weight_stability/edge_weights_stability_train_2-decim_graph_{i}.csv")

#### Visualize results of edge-weights stability

In [ ]:
dir_ = f"../data/bootstrap_results/edge_weight_stability/"

stability_estimates = []

for file in sorted(glob.glob(dir_ + "*_2-decim*.pkl")):
    print(file)
    df = pd.read_pickle(file)
    stability_estimates.append(df)

In [ ]:
for index, stability in enumerate(zip(stability_estimates, orders)):
    stability_estimate = stability[0]
    order = stability[1]
    
    df = stability_estimate.copy()

    labels = df.copy().to_numpy()
    values = df.copy().to_numpy() 

    n = df.shape[0]
    upper_triangle = np.triu(np.ones((n, n), dtype=bool), k=1)
    values[upper_triangle] = 0

    df_with_values = pd.DataFrame(values, columns=df.columns, index=df.index).astype(float)

    cm = 1/2.54
    dpi = 500

    sns.set_style("white")
    plt.rcParams['figure.dpi'] = dpi
    plt.rcParams['figure.figsize'] = [15*cm,15*cm]
    plt.rcParams["font.size"] = 4
    plt.rcParams['ytick.labelsize'] = 6
    plt.rcParams['xtick.labelsize'] = 6
    plt.rcParams['axes.labelsize'] = 7
    plt.rcParams["axes.edgecolor"] = ".15"
    plt.rcParams["axes.linewidth"]  = 0.5

    cmap = sns.diverging_palette(100, 7, s=75, l=40,
                        n=5, center="light", as_cmap=True)

    g = sns.heatmap(
    data = df_with_values,
    annot=labels,
    center=0.0,
    fmt = '',
    square=True,
    cmap=cmap,
    cbar=False,
    linewidths=0.3,
    )
    
    g.figure.savefig(f"../data/bootstrap_results/edge_weight_stability/edge_weigh_stability_measures_dpi_{dpi}_graph_{index}.png",  bbox_inches='tight')

    plt.show()

## Test edge-weight differences

In [ ]:
# Function to extract node-node pairs from matrix columns
def extract_node_pairs(matrix):
    n = matrix.shape[0]
    node_pairs = []
    for i in range(n):
        for j in range(i+1, n):
            node_pairs.append((matrix.columns[i], matrix.columns[j]))
    return node_pairs

# Extract node-node pairs from the first matrix
node_pairs = extract_node_pairs(bootstrap_precisions[0])

# Function to calculate pairwise differences in edge weights for each matrix
def calculate_links_values(matrices_list):
    pairwise_diffs_all_matrices = {}
    for matrix in matrices_list:
        n = matrix.shape[0]
        for i in range(n):
            for j in range(i+1, n):
                value = matrix[i, j]
                pair = (columns[i], columns[j])
                if pair not in pairwise_diffs_all_matrices:
                    pairwise_diffs_all_matrices[pair] = []
                pairwise_diffs_all_matrices[pair].append(value)
    return pairwise_diffs_all_matrices

### 2.1 Pair-wise differences between edge weights within the graph

In [ ]:
# Function to perform statistical tests for each unique pair of differences
def perform_statistical_tests(pairwise_diffs_all_matrices):
    for i, node_pair1 in enumerate(node_pairs):
        for j, node_pair2 in enumerate(node_pairs):
            if i >= j:
                continue  # We only need to perform tests once for each pair
            diffs_pair1 = pairwise_diffs_all_matrices.get(node_pair1, [])
            diffs_pair2 = pairwise_diffs_all_matrices.get(node_pair2, [])
            _, p_value = ttest_ind(diffs_pair1, diffs_pair2)
            p_values_matrix[i, j] = p_value
            p_values_matrix[j, i] = p_value  # Symmetric matrix
    return p_values_matrix

In [ ]:
for index, (bootstrap_precisions, order) in enumerate(zip(precision_bootstrap_matrices, orders)):

    # reorder and rename nodes
    bootstrap_precisions = [reorder_matrix(matrix, matrix.columns, matrix.columns, mapping=mapping, mapping_order=order, to_numpy=False) for matrix in bootstrap_precisions]
    matrices_list = [np.array(matrix) for matrix in bootstrap_precisions]
    columns = bootstrap_precisions[0].columns
    
    # Calculate pairwise differences for each matrix
    pairwise_diffs_all_matrices = calculate_links_values(matrices_list)
    
    # Create a matrix to store the p-values for statistical tests
    num_nodes = len(node_pairs)
    p_values_matrix = np.zeros((num_nodes, num_nodes))
  
    # Perform statistical tests
    p_values_matrix = perform_statistical_tests(pairwise_diffs_all_matrices)
    p_values_matrix_df = pd.DataFrame(p_values_matrix, columns =node_pairs, index=node_pairs)
    
    # save results
    p_values_matrix_df.to_pickle(f'../data/bootstrap_results/edge-weight_diff_test_graph_{index}.pkl')
    p_values_matrix_df.to_csv(f'../data/bootstrap_results/edge-weight_diff_test_graph_{index}.pkl')

### 2.2 Pair-wise differences between edge weights between models

In [ ]:
def perform_statistical_tests_between(m1_links, m2_links, node_pairs_1, node_pairs_2):
    if len(node_pairs_1) != len(node_pairs_2):
        node_pairs = node_pairs_m1 if len(node_pairs_m1) <= len(node_pairs_m2) else node_pairs_m2
        num_nodes = len(node_pairs)
   
        p_values_matrix = np.full((num_nodes, num_nodes), np.nan)

        for i, node_pair1 in enumerate(node_pairs):
            for j, node_pair2 in enumerate(node_pairs):
                if i > j:
                    continue 
                diffs_pair1 = m1_links.get(node_pair1, [])
                diffs_pair2 = m2_links.get(node_pair2, [])
                _, p_value = ttest_ind(diffs_pair1, diffs_pair2)
                p_values_matrix[i, j] = p_value
                p_values_matrix[j, i] = p_value
                p_values_matrix_df = pd.DataFrame(p_values_matrix, columns=node_pairs, index=node_pairs)
    else:
        num_nodes = len(node_pairs_1)
        
        p_values_matrix = np.full((num_nodes, num_nodes), np.nan)

        for i, node_pair1 in enumerate(node_pairs_1):
            for j, node_pair2 in enumerate(node_pairs_2):
                diffs_pair1 = m1_links.get(node_pair1, [])
                diffs_pair2 = m2_links.get(node_pair2, [])
                _, p_value = ttest_ind(diffs_pair1, diffs_pair2)
                p_values_matrix[i, j] = p_value
                p_values_matrix_df = pd.DataFrame(p_values_matrix, columns=node_pairs_2, index=node_pairs_1)
        
    return p_values_matrix_df

def compare_edge_weights(
    bootstrap_matrices_1,
    bootstrap_matrices_2,
    save=False
):
    bootstrap_matrices_1_list = [np.array(matrix) for matrix in bootstrap_matrices_1]
    bootstrap_matrices_2_list = [np.array(matrix) for matrix in bootstrap_matrices_2]
    
    # extract links from precision matrices
    m1_links = calculate_links_values(bootstrap_matrices_1_list, bootstrap_matrices_1[0].columns)
    m2_links = calculate_links_values(bootstrap_matrices_2_list, bootstrap_matrices_2[0].columns)
    
    # calculate list of links (node pairs)
    node_pairs_m1 = extract_node_pairs(m_1[0])
    node_pairs_m2 = extract_node_pairs(m_2[0])
    
    p_values_matrix_df = perform_statistical_tests_between(m1_links, m2_links, node_pairs_m1, node_pairs_m2)

    if save != False:
        p_values_matrix_df.to_csv(f'../data/bootstrap_results/models_statistical_diffs/{save}.csv')

    return p_values_matrix_df 

- Calculate statistical difference between ERN and ERN-COV models

In [ ]:
# reorder and change names of the nodes
m_1 = [reorder_matrix(
        matrix, 
        precision_bootstrap_matrices[0][0].columns, 
        precision_bootstrap_matrices[0][0].columns, 
        mapping=mapping, 
        mapping_order=ern_order,
        to_numpy=False
    ) for matrix in precision_bootstrap_matrices[0]
]

# reorder and change names of the nodes
m_2 = [
    reorder_matrix(
        matrix, 
        precision_bootstrap_matrices[1][0].columns, 
        precision_bootstrap_matrices[1][0].columns, 
        mapping=mapping, 
        mapping_order=ern_cov_order,
        to_numpy=False
    ) for matrix in precision_bootstrap_matrices[1]
]

p_values_matrix = compare_edge_weights(
    m_1, 
    m_2,
    save=False
)

- Calculate statistical difference between CRN and CRN-COV models

In [ ]:
# reorder and change names of the nodes
m_1 = [reorder_matrix(
        matrix, 
        precision_bootstrap_matrices[2][0].columns, 
        precision_bootstrap_matrices[2][0].columns, 
        mapping=mapping, 
        mapping_order=crn_order,
        to_numpy=False
    ) for matrix in precision_bootstrap_matrices[2]
]

# reorder and change names of the nodes
m_2 = [
    reorder_matrix(
        matrix, 
        precision_bootstrap_matrices[3][0].columns, 
        precision_bootstrap_matrices[3][0].columns, 
        mapping=mapping, 
        mapping_order=crn_cov_order,
        to_numpy=False
    ) for matrix in precision_bootstrap_matrices[3]
]

p_values_matrix = compare_edge_weights(
    m_1, 
    m_2,
    save=False
)

- Calculate statistical difference between ERN and CRN models

In [ ]:
# reorder and change names of the nodes
m_1 = [reorder_matrix(
        matrix, 
        precision_bootstrap_matrices[0][0].columns, 
        precision_bootstrap_matrices[0][0].columns, 
        mapping=mapping, 
        mapping_order=ern_order,
        to_numpy=False
    ) for matrix in precision_bootstrap_matrices[0]
]

# reorder and change names of the nodes
m_2 = [
    reorder_matrix(
        matrix, 
        precision_bootstrap_matrices[2][0].columns, 
        precision_bootstrap_matrices[2][0].columns, 
        mapping=mapping, 
        mapping_order=crn_order,
        to_numpy=False
    ) for matrix in precision_bootstrap_matrices[2]
]

p_values_matrix = compare_edge_weights(
    m_1, 
    m_2,
    save=False
)

- Calculate statistical difference between ERN-COV and CRN-COV models

In [ ]:
# reorder and change names of the nodes
m_1 = [reorder_matrix(
        matrix, 
        precision_bootstrap_matrices[1][0].columns, 
        precision_bootstrap_matrices[1][0].columns, 
        mapping=mapping, 
        mapping_order=ern_cov_order,
        to_numpy=False
    ) for matrix in precision_bootstrap_matrices[1]
]

# reorder and change names of the nodes
m_2 = [
    reorder_matrix(
        matrix, 
        precision_bootstrap_matrices[3][0].columns, 
        precision_bootstrap_matrices[3][0].columns, 
        mapping=mapping, 
        mapping_order=crn_cov_order,
        to_numpy=False
    ) for matrix in precision_bootstrap_matrices[3]
]

p_values_matrix = compare_edge_weights(
    m_1, 
    m_2,
    save=False
)

## 3. Sensitivity and stability of networks

In [ ]:
def binarize_matrix(matrix, threshold=0.02):
    return np.where(np.abs(matrix) < threshold, 0, 1)

def calculate_specificity_for_element(true_data, estimated_matrices, i, j):
    tp = np.sum([1 for matrix in estimated_matrices if matrix[i, j] == 1 and true_data[i, j] == 1])
    tn = np.sum([1 for matrix in estimated_matrices if matrix[i, j] == 0 and true_data[i, j] == 0])
    fp = np.sum([1 for matrix in estimated_matrices if matrix[i, j] == 1 and true_data[i, j] == 0])
    fn = np.sum([1 for matrix in estimated_matrices if matrix[i, j] == 0 and true_data[i, j] == 1])

    specificity = tn / (tn + fp) if (tn + fp) != 0 else 0  # Handling division by zero

    return specificity

def calculate_sensitivity_for_element(true_data, estimated_matrices, i, j):
    tp = np.sum([1 for matrix in estimated_matrices if matrix[i, j] == 1 and true_data[i, j] == 1])
    tn = np.sum([1 for matrix in estimated_matrices if matrix[i, j] == 0 and true_data[i, j] == 0])
    fp = np.sum([1 for matrix in estimated_matrices if matrix[i, j] == 1 and true_data[i, j] == 0])
    fn = np.sum([1 for matrix in estimated_matrices if matrix[i, j] == 0 and true_data[i, j] == 1])

    sensitivity = tp / (tp + fn) if (tp + fn) != 0 else 0  # Handling division by zero

    return sensitivity

def plot_sensitivity_specificity_results(
    sensitivity_df, 
    specificity_df,
    save=False
):
    specificity_df = specificity_df.set_index('Unnamed: 0')
    sensitivity_df = sensitivity_df.set_index('Unnamed: 0')
    
    specificity_mask = np.triu(np.where(specificity_df == 0, False, True), k=1)
    sensitivity_mask = np.tril(np.where(sensitivity_df == 0, False, True), k=-1)
    
    
    # plotting parameters
    cmap = sns.diverging_palette(100, 7, s=75, l=40,n=5, center="light", as_cmap=True)
    cm = 1/2.54
    dpi = 500
    
    sns.set_style("white")
    plt.rcParams['figure.dpi'] = dpi
    plt.rcParams['figure.figsize'] = [15*cm,15*cm]
    plt.rcParams["font.size"] = 4
    plt.rcParams['ytick.labelsize'] = 6
    plt.rcParams['xtick.labelsize'] = 6
    plt.rcParams['axes.labelsize'] = 7
    plt.rcParams["axes.edgecolor"] = ".15"
    plt.rcParams["axes.linewidth"]  = 0.5
    
    # create canvas
    figure, axis = plt.subplots(1, 1)

    # plot sensitivity
    g = sns.heatmap(
        data = sensitivity_df,
        annot=True,
        mask = np.invert(sensitivity_mask),
        center=0.5,
        fmt='.2f',
        square=True,
        cmap=cmap,
        cbar=False,
        linewidths=1,
        ax=axis
    )

    # plot specificity
    g_2 = sns.heatmap(
        data = specificity_df,
        annot=True,
        mask = np.invert(specificity_mask),
        center=0.5,
        fmt='.2f',
        square=True,
        cmap=cmap,
        cbar=False,
        linewidths=1,
        ax=axis
    )

    if save != False:
        figure.savefig(f"../data/bootstrap_results/specificity_sensitivity/{save}.png",  bbox_inches='tight')
    
    
    plt.show()
    return figure    

Read original models

In [ ]:
ern_model_df = pd.read_pickle('../data/precision_matrixes/train/precision_matrix_0_ntrials_1000_sklearn_cv3_th0.65_without_skewd.pkl')
ern_cov_model_df = pd.read_pickle('../data/precision_matrixes/train/precision_matrix_1_ntrials_1000_sklearn_cv3_th0.65_without_skewd_no_sex.pkl')
crn_model_df = pd.read_pickle('../data/precision_matrixes/train/precision_matrix_2_ntrials_1000_sklearn_cv3_th0.65_without_skewd.pkl')
crn_cov_model_df = pd.read_pickle('../data/precision_matrixes/train/precision_matrix_3_ntrials_1000_sklearn_cv3_th0.65_without_skewd_no_sex.pkl')

precision_matrixes = [
    ern_model_df,
    ern_cov_model_df,
    crn_model_df,
    crn_cov_model_df,
]

Read bootstrap results

In [ ]:
dir_ = f"data/bootstrap_results/"

precision_bootstrap_matrices = []

for file in sorted(glob.glob(dir_ + "*.pkl")):
    print(file)
    precision_bootstrap_matrix = read_from_pickle(file)
    precision_bootstrap_matrices.append(precision_bootstrap_matrix)

Calculate sensitivity ans specificity items for each node

In [ ]:
for index, (original_model, bootstrap_precisions, order) in enumerate(zip(precision_matrixes, precision_bootstrap_matrices, orders)):

    # reorder and rename nodes
    bootstrap_precisions = [reorder_matrix(matrix, bootstrap_precisions[0].columns, bootstrap_precisions[0].columns, mapping=mapping, mapping_order=order) for matrix in bootstrap_precisions]
    original_model_df = reorder_matrix(original_model, original_model.columns, original_model.columns, mapping=mapping, mapping_order=order)
    
    original_model = np.array(binarize_matrix(original_model_df))
    matrices_list = [np.array(binarize_matrix(matrix)) for matrix in bootstrap_precisions]

    specificity_matrix = np.zeros_like(original_model, dtype=float)
    sensitivity_matrix = np.zeros_like(original_model, dtype=float)

    for i in range(original_model.shape[0]):
        for j in range(original_model.shape[1]):
            specificity_matrix[i, j] = calculate_specificity_for_element(original_model, matrices_list, i, j)
            sensitivity_matrix[i, j] = calculate_sensitivity_for_element(original_model, matrices_list, i, j)
    
    specificity_df = pd.DataFrame(specificity_matrix, columns=original_model_df.columns, index=original_model_df.columns)
    sensitivity_df = pd.DataFrame(sensitivity_matrix, columns=original_model_df.columns, index=original_model_df.columns)

    specificity_df.to_csv(f'../data/bootstrap_results/specificity_sensitivity/specificity_model_{index}.csv')
    sensitivity_df.to_csv(f'../data/bootstrap_results/specificity_sensitivity/sensitivity_model_{index}.csv')

#### Visualize sensitivity and stability

In [ ]:
dir_ = f"../data/bootstrap_results/specificity_sensitivity/"

sensitivity_results = []

for file in sorted(glob.glob(dir_ + "sensitivity*.csv")):
    print(file)
    result = pd.read_csv(file)
    sensitivity_results.append(result)

specificity_results = []

for file in sorted(glob.glob(dir_ + "specificity*.csv")):
    print(file)
    result = pd.read_csv(file)
    specificity_results.append(result)

- ERN model

In [ ]:
ern_specificity_df = specificity_results[0]
ern_sensitivity_df = sensitivity_results[0]

_ = plot_sensitivity_specificity_results(
    sensitivity_df = ern_sensitivity_df,
    specificity_df = ern_specificity_df,
    save=False,
)

- ERN cov model

In [ ]:
ern_specificity_df = specificity_results[1]
ern_sensitivity_df = sensitivity_results[1]

_ = plot_sensitivity_specificity_results(
    sensitivity_df = ern_sensitivity_df,
    specificity_df = ern_specificity_df,
    save=False,
)

- CRN model

In [ ]:
crn_specificity_df = specificity_results[2]
crn_sensitivity_df = sensitivity_results[2]

_ = plot_sensitivity_specificity_results(
    sensitivity_df = crn_sensitivity_df,
    specificity_df = crn_specificity_df,
    save=False,
)

- CRN cov model

In [ ]:
crn_specificity_df = specificity_results[3]
crn_sensitivity_df = sensitivity_results[3]

_ = plot_sensitivity_specificity_results(
    sensitivity_df = crn_sensitivity_df,
    specificity_df = crn_specificity_df,
    save=False,
)

## 3. Network Measures

In [ ]:
def get_links(precision_matrix_df, threshold=0.02):
    precision_matrix_df = precision_matrix_df.where(np.triu(np.ones(precision_matrix_df.shape)).astype(bool))
    
    links = precision_matrix_df.stack().reset_index()
    links.columns = ['var1', 'var2','weight']
    links=links.loc[ (abs(links['weight']) > threshold) &  (links['var1'] != links['var2']) ]
        
    links = links.round(3)
    
    return links

In [ ]:
def read_from_pickle(path):
    with open(path, 'rb') as file:
        precision_matrices = pickle.load(file)
    return precision_matrices

In [ ]:
def get_graph(precision_matrix_df):
    this_links = get_links(precision_matrix_df)
    this_links['weight'] = this_links['weight'].abs()
    G_ = nx.from_pandas_edgelist(this_links,'var1','var2', edge_attr='weight', create_using=nx.Graph())

    return G_

def get_bootstrap_network_measures(precision_bootstrap_matrices, dataset_train, dataset_test):
    network_analysis_results_df = pd.DataFrame()

    for bootstrap_precision_matrix in precision_bootstrap_matrices:
        bootstrap_graph = get_graph(bootstrap_precision_matrix)
        
        measures = [
                (calculate_nodes_predictability, {'X': dataset_test}), 
                (nx.degree_centrality, {}),
                (nx.closeness_centrality, {}),
                (current_flow_closeness_centrality, {'weight': 'weight'}),
                (nx.betweenness_centrality, {'weight': 'weight'}),
                (nx.current_flow_betweenness_centrality, {'weight': 'weight'}),
                (nx.load_centrality, {})
        ]
        
        for measure, measure_parameters in measures:
            G = copy.deepcopy(bootstrap_graph)
            network_measure = measure(G = G, **measure_parameters)
            network_measure = {k: v for k, v in sorted(network_measure.items(), key=lambda item: item[0], reverse=True)}
            network_measure_nodes = network_measure.keys()
            network_measure_values = network_measure.values()
            network_measure_df = pd.DataFrame({
                'node': network_measure_nodes,
                'value' : network_measure_values,
                'measure' : [measure.__name__] * len(network_measure_nodes),
            })
            
            network_analysis_results_df = pd.concat([network_analysis_results_df, network_measure_df], ignore_index = True)

    return network_analysis_results_df

Define custrom network measures

In [ ]:
def calculate_nodes_predictability(X, G):
    explained_variance = dict()
        
    for node in G.nodes():
        y_ = X[[node]]

        neighbors = list(G.neighbors(node))

        X_ = X.loc[:, neighbors]

        lm = LinearRegression()
        lm.fit(X_, y_)

        score = lm.score(X_,y_)
        explained_variance[node] = score

    return explained_variance

In [ ]:
def current_flow_closeness_centrality(G, weight):
    G_ = copy.deepcopy(G.copy())
    for u, v, data in G_.edges(data=True):
        data['weight'] = abs(data['weight'])
        
    measure = nx.current_flow_closeness_centrality(G_, weight=weight)
    return measure

### Test if there are differences between network measures based on the bootstrapped samples

Read bootstrap results

In [ ]:
dir_ = f"../data/bootstrap_results/"

precision_bootstrap_matrices = []

for file in sorted(glob.glob(dir_ + "*.pkl")):
    print(file)
    precision_bootstrap_matrix = read_from_pickle(file)
    precision_bootstrap_matrices.append(precision_bootstrap_matrix)

Calculate network measures on bootstrapped samples and save the results

In [ ]:
for index, (bootstrap_precision_matrix, dataset_train, dataset_test) in enumerate(zip(precision_bootstrap_matrices, datasets_train, datasets_test)):
    model_network_analysis_results_df = get_bootstrap_network_measures(bootstrap_precision_matrix, dataset_train, dataset_test)
    model_network_analysis_results_df.to_pickle(f'../data/bootstrap_results/network_measures_diffs/model_{index}.pkl')

#### 3.1 Calculate if differences between network measures within one network are significant

In [ ]:
# Read network measures estimated on bootstrap samples
dir_ = f"../data/bootstrap_results/network_measures_diffs/"

models_bootstrap_network_measures = []

for file in sorted(glob.glob(dir_ + "model*.pkl")):
    print(file)
    bootstrap_network_measures = read_from_pickle(file)
    models_bootstrap_network_measures.append(bootstrap_network_measures)

In [ ]:
# Define measures to test
measures = ['calculate_nodes_predictability', 'degree_centrality', 'current_flow_closeness_centrality']

Perform statistical tests

In [ ]:
for measure in measures:
    for index, (model_bootstrap_results, data, order) in enumerate(zip(models_bootstrap_network_measures, datasets_train, orders)):
        nodes_list = data.columns
        results_df = pd.DataFrame(np.nan, columns=nodes_list, index=nodes_list)
    
        for node1 in nodes_list:
            for node2 in nodes_list:
                if node1 != node2:
                    node1_values = model_bootstrap_results[
                        (model_bootstrap_results['node'] == node1) & 
                        (model_bootstrap_results['measure'] == measure)
                    ]['value'].to_numpy()
        
                    node2_values = model_bootstrap_results[
                        (model_bootstrap_results['node'] == node2) & 
                        (model_bootstrap_results['measure'] == measure)
                    ]['value'].to_numpy()
        
                    _, p_value = ttest_rel(node1_values, node2_values)
                    results_df.loc[node1, node2] = p_value
                
                if node1 == node2:
                    node_value = model_bootstrap_results[
                        (model_bootstrap_results['node'] == node1) & 
                        (model_bootstrap_results['measure'] == measure)
                    ]['value'].to_numpy().mean()
        
                    results_df.loc[node1, node2] = node_value

        results_df = reorder_matrix(
            results_df, 
            results_df.columns, 
            results_df.columns, 
            mapping=mapping, 
            mapping_order=order, 
            to_numpy=False
        )
        results_df.to_csv(f'../data/bootstrap_results/network_measures_diffs/{measure}_stats_tests_model_{index}.csv')

#### 3.2 Calculate if differences between network measures between networks are significant

In [ ]:
def perform_statistical_tests_network_measures(
    model_1_bootstrap_results, 
    model_2_bootstrap_results, 
    nodes_list_1, 
    nodes_list_2, 
    measure,
    mapping, 
    order_columns,
    order_index,
):
    if len(nodes_list_1) != len(nodes_list_2):
        nodes_list = nodes_list_1 if len(nodes_list_1) < len(nodes_list_2) else nodes_list_2
        results_df = pd.DataFrame(np.nan, columns=nodes_list, index=nodes_list)

        for node1 in nodes_list:
            for node2 in nodes_list:
                node1_values = model_1_bootstrap_results[
                    (model_1_bootstrap_results['node'] == node1) & 
                    (model_1_bootstrap_results['measure'] == measure)
                ]['value'].to_numpy()
                
                node2_values = model_2_bootstrap_results[
                    (model_2_bootstrap_results['node'] == node2) & 
                    (model_2_bootstrap_results['measure'] == measure)
                ]['value'].to_numpy()
                
                _, p_value = ttest_rel(node1_values, node2_values)
                results_df.loc[node1, node2] = p_value

    else:
        results_df = pd.DataFrame(np.nan, columns=nodes_list_1, index=nodes_list_2)

        for node1 in nodes_list_1:
            for node2 in nodes_list_2:
                node1_values = model_1_bootstrap_results[
                    (model_1_bootstrap_results['node'] == node1) & 
                    (model_1_bootstrap_results['measure'] == measure)
                ]['value'].to_numpy()
                
                node2_values = model_2_bootstrap_results[
                    (model_2_bootstrap_results['node'] == node2) & 
                    (model_2_bootstrap_results['measure'] == measure)
                ]['value'].to_numpy()
                
                _, p_value = ttest_ind(node1_values, node2_values)
                results_df.loc[node2, node1] = p_value
    
    results_df = reorder_matrix(
        results_df, 
        results_df.columns, 
        results_df.index, 
        mapping=mapping, 
        mapping_order_columns=order_columns, 
        mapping_order_index=order_index,
        to_numpy=False
    )
    
    return results_df

- ERN and ERN-COV

In [ ]:
model_1_bootstrap_results = models_bootstrap_network_measures[0]
model_2_bootstrap_results = models_bootstrap_network_measures[1]

nodes_list_1 = datasets_train[0].columns
nodes_list_2 = datasets_train[1].columns

for measure in measures:
    network_measures_diff_results_df = perform_statistical_tests_network_measures(
        model_1_bootstrap_results, 
        model_2_bootstrap_results, 
        nodes_list_1, 
        nodes_list_2, 
        measure,
        mapping, 
        ern_order,
        ern_order,
    )
    
    network_measures_diff_results_df.to_csv(f'../data/bootstrap_results/network_measures_diffs/{measure}_stats_tests_ern_ern-cov.csv')

- CRN and CRN-COV

In [ ]:
model_1_bootstrap_results = models_bootstrap_network_measures[2]
model_2_bootstrap_results = models_bootstrap_network_measures[3]

nodes_list_1 = datasets_train[2].columns
nodes_list_2 = datasets_train[3].columns

for measure in measures:
    network_measures_diff_results_df = perform_statistical_tests_network_measures(
        model_1_bootstrap_results, 
        model_2_bootstrap_results, 
        nodes_list_1, 
        nodes_list_2, 
        measure,
        mapping, 
        crn_order,
        crn_order,
    )
    
    network_measures_diff_results_df.to_csv(f'../data/bootstrap_results/network_measures_diffs/{measure}_stats_tests_crn_crn-cov.csv')

- ERN and CRN

In [ ]:
model_1_bootstrap_results = models_bootstrap_network_measures[0]
model_2_bootstrap_results = models_bootstrap_network_measures[2]

nodes_list_1 = datasets_train[0].columns
nodes_list_2 = datasets_train[2].columns

for measure in measures:
    network_measures_diff_results_df = perform_statistical_tests_network_measures(
        model_1_bootstrap_results, 
        model_2_bootstrap_results, 
        nodes_list_1, 
        nodes_list_2, 
        measure,
        mapping, 
        ern_order,
        crn_order,
    )
    
    network_measures_diff_results_df.to_csv(f'../data/bootstrap_results/network_measures_diffs/{measure}_stats_tests_ern_crn.csv')

- ERN-COV and CRN-COV

In [ ]:
model_1_bootstrap_results = models_bootstrap_network_measures[1]
model_2_bootstrap_results = models_bootstrap_network_measures[3]

nodes_list_1 = datasets_train[1].columns
nodes_list_2 = datasets_train[3].columns

for measure in measures:
    network_measures_diff_results_df = perform_statistical_tests_network_measures(
        model_1_bootstrap_results, 
        model_2_bootstrap_results, 
        nodes_list_1, 
        nodes_list_2, 
        measure,
        mapping, 
        ern_cov_order,
        crn_cov_order,
    )
    
    network_measures_diff_results_df.to_csv(f'../data/bootstrap_results/network_measures_diffs/{measure}_stats_tests_ern-cov_crn-cov.csv')

## 4.Stability of network measures

### Visualize results of network measures stability tests

In [ ]:
dir_ = f"data/network_analysis/stability_estimates/"

ranked_stability_estimates = []

for file in sorted(glob.glob(dir_ + "*.pkl")):
    print(file)
    stability_estimate = pd.read_pickle(file)
    ranked_stability_estimates.append(stability_estimate)

In [ ]:
for index, stability_estimate in enumerate(ranked_stability_estimates):
    with pd.option_context('display.max_colwidth', None, 'display.max_columns', None,  'display.max_rows', None,):
        print(f'GRAPH:   ------ {index}')
        display(stability_estimate.groupby(['measure', 'level']).describe())

In [ ]:
def draw_network_measures_stability_estimates(stability_estimates_df, exclude=None):
    cm = 1/2.54
    dpi = 500

    sns.set_style("whitegrid")

    plt.rcParams['figure.dpi'] = dpi
    plt.rcParams['figure.figsize'] = [10*cm,6*cm]
    plt.rcParams['ytick.labelsize'] = 6
    plt.rcParams['xtick.labelsize'] = 6
    plt.rcParams['axes.labelsize'] = 7
    plt.rcParams["axes.edgecolor"] = ".15"
    plt.rcParams["axes.linewidth"]  = 0.5
    
    stability_estimates_filtered_df = stability_estimates_df[stability_estimates_df['measure'] != exclude]

    g = sns.lineplot(
        data=stability_estimates_filtered_df, 
        x='level', 
        y='similatity', 
        hue='measure',
        palette='colorblind',
    )
    g.invert_xaxis()
    g.set(xlabel='Sampled cases (%)', ylabel='Average correlation with original sample')
    g.set(ylim=(-1.1, 1.1))

    sns.move_legend(
        g, "lower center",
        bbox_to_anchor=(.5, 1), ncol=3, title=None, frameon=False, fontsize=6,
    )

    new_labels = labels=['predictability', 'd-centrality', 'closeness', 'c-f-closeness', 'betweenness', 'c-f-betweenness']
    for t, l in zip(g.legend_.texts, new_labels):
        t.set_text(l)

    plt.axhline(y=0.0, color='black', linestyle='-', linewidth=0.3)


    plt.show()


# g.figure.savefig(f"data/n_estimators/network_stability_measures_dpi_{dpi}.png",  bbox_inches='tight')

In [ ]:
for stability_estimate in ranked_stability_estimates:
    draw_network_measures_stability_estimates(stability_estimate, 'newman_betweenness_centrality')

## 5. Test of similarity of networks estimated on train and test sets

In [ ]:
def get_matrix_triu_as_list_adjacency(matrix, threshold = 0.002):
    
    precision_matrix_df = matrix.where(np.triu(np.ones(matrix.shape)).astype(dtype=bool))
    precision_matrix_df = precision_matrix_df.mask(abs(precision_matrix_df) <= threshold, 0.0)
    
    links = precision_matrix_df.stack().reset_index()
    links.columns = ['var1', 'var2','weight']
    links=links.loc[(links['var1'] != links['var2']) ]
    
    links = links['weight'].to_numpy()
    
    links[abs(links) > 0] = 1.0
    
    return links

In [ ]:
def get_matrix_triu_as_list(matrix, threshold = 0.002):
    
    precision_matrix_df = matrix.where(np.triu(np.ones(matrix.shape)).astype(dtype=bool))
    precision_matrix_df = precision_matrix_df.mask(abs(precision_matrix_df) <= threshold, 0.0)
    
    links = precision_matrix_df.stack().reset_index()
    links.columns = ['var1', 'var2','weight']
    links=links.loc[(links['var1'] != links['var2']) ]
    
    links = links['weight'].to_numpy()
        
    return links

In [ ]:
def pearsonr_links_similarity(matrix1, matrix2, adjacency=True):
    links1 = get_matrix_triu_as_list_adjacency(matrix1) if adjacency else get_matrix_triu_as_list(matrix1)
    links2 = get_matrix_triu_as_list_adjacency(matrix2) if adjacency else get_matrix_triu_as_list(matrix2)
        
    corr, p_value = pearsonr(links1, links2)
    
    return corr, p_value

In [ ]:
def hamming_distance_similarity(matrix1, matrix2, adjacency=True):
    links1 = get_matrix_triu_as_list_adjacency(matrix1) if adjacency else get_matrix_triu_as_list(matrix1)
    links2 = get_matrix_triu_as_list_adjacency(matrix2) if adjacency else get_matrix_triu_as_list(matrix2)
        
    h_sim = 1 - distance.hamming(links1, links2)
    
    return h_sim

Read data

In [ ]:
# 0 for ERN, 1 for ERN cov, 2 for CRN, 3 for CRN cov
index = 0
model_train = pd.read_pickle(f"data/precision_matrixes/train/precision_matrix_{index}_ntrials_1000_sklearn_cv3_th0.65_without_skewd.pkl")
model_test = pd.read_pickle(f"data/precision_matrixes/test/precision_matrix_{index}_ntrials_1000_sklearn_cv3_th0.65_without_skewd.pkl")

1. similarity between the train ans test nets (as adjencacy matrices)

In [ ]:
print(f'Pearson\'s r coef: {pearsonr_links_similarity(model_train, model_test)[0]}, p-value: {pearsonr_links_similarity(model_train, model_test)[1]}')
print(f'Hamming distance: {hamming_distance_similarity(model_train, model_test)}')

2. similarity between train and test nets (as weighted matrices)

In [ ]:
print(f'Pearson\'s r coef: {pearsonr_links_similarity(model_train, model_test, adjacency=False)[0]}, p-value: {pearsonr_links_similarity(model_train, model_test, adjacency=False)[1]}')
print(f'Hamming distance: {hamming_distance_similarity(model_train, model_test, adjacency=False)}')

3. similarity of only ERN associations between train and test sets (weights matrices)

In [ ]:
# node = 'e_CRN' || 'e_ERN'
node = 'e_ERN'

- adjacency

In [ ]:
train_ = model_train[node].to_numpy()
test_ = model_test[node].to_numpy()

train_ = [0 if abs(item) <= 0.02 else 1 for item in train_]
test_ = [0 if abs(item) <= 0.02 else 1 for item in test_]

In [ ]:
print(f'Pearson\'s r: {scipy.stats.pearsonr(train_, test_)}')
print(f'Hamming distance: {1 - distance.hamming(train_, test_)}')

- weighten

In [ ]:
train_ = model_train[node].to_numpy()
test_ = model_test[node].to_numpy()

print(f'Pearson\'s r: {scipy.stats.pearsonr(train_, test_)}')
print(f'Hamming distance: {1 - distance.hamming(train_, test_)}')

## 6. Test wether differences between covariates models and no-covariates models are matter of power

In [ ]:
score_metric="log_likelihood"
n_trials = 1000
alphas = np.linspace(0.01, 0.1, 20)
cv = 3
threshold = 0.65

In [ ]:
datasets = [
    ern_data_df, 
    crn_data_df,
]

In [ ]:
columns_to_drop = ['WASH', 'NEU']

In [ ]:
random_data_df = pd.DataFrame({
    'R1': np.random.rand(datasets_no_skewed_scales[0].shape[0]),
    'R2': np.random.rand(datasets_no_skewed_scales[0].shape[0]),
    'R3': np.random.rand(datasets_no_skewed_scales[0].shape[0]),
    'R4': np.random.rand(datasets_no_skewed_scales[0].shape[0]),
})
random_data_df.head()

In [ ]:
datasets_no_skewed_scales = []
for model in datasets:
    this_dataset = model.drop(columns=columns_to_drop)
    this_dataset = pd.concat([this_dataset, random_data_df], axis=1)
    datasets_no_skewed_scales.append(this_dataset)

In [ ]:
if dataset == 'test':
    print(f"Test dataset")
    precision_matrixes_test = []
    covariance_matrixes_test = []
    support_matrixes_test = []
    proportion_matrixes_test = []
    average_estimators_test = []
    
    for model in datasets_no_skewed_scales:
        covariance_matrix_df, precision_matrix_df, support_matrix_df, proportion_matrix_df, estimator = learn_graph_structure_adaptive_average_sklearn(
            model, 
            penalization='random',
            n_trials=n_trials,
            score_metric=score_metric,
            cv=cv,
            lam=None,
            alphas=alphas,
            threshold=threshold,
        )
        precision_matrixes_test.append(precision_matrix_df) 
        covariance_matrixes_test.append(covariance_matrix_df)
        support_matrixes_test.append(support_matrix_df)
        proportion_matrixes_test.append(proportion_matrix_df)
        average_estimators_test.append(estimator)

else:
    print(f"Train dataset")
    precision_matrixes = []
    covariance_matrixes = []
    support_matrixes = []
    proportion_matrixes = []
    average_estimators = []

    for model in datasets_no_skewed_scales:
        covariance_matrix_df, precision_matrix_df, support_matrix_df, proportion_matrix_df, estimator  = learn_graph_structure_adaptive_average_sklearn(
            model, 
            penalization='random',
            n_trials=n_trials,
            score_metric=score_metric,
            cv=cv,
            lam=None,
            alphas=alphas,
            threshold=threshold,
        )
        precision_matrixes.append(precision_matrix_df)
        covariance_matrixes.append(covariance_matrix_df)
        support_matrixes.append(support_matrix_df)
        proportion_matrixes.append(proportion_matrix_df)
        average_estimators.append(estimator)

In [ ]:
precision_matrixes = []

for i in range(0,50):
    random_data_df = pd.DataFrame({
        'R1': np.random.normal(scale=1,size=datasets_no_skewed_scales[0].shape[0]),
        'R2': np.random.normal(scale=1,size=datasets_no_skewed_scales[0].shape[0]),
        'R3': np.random.normal(scale=1,size=datasets_no_skewed_scales[0].shape[0]),
        'R4': np.random.normal(scale=1,size=datasets_no_skewed_scales[0].shape[0]),
    })

    datasets_no_skewed_scales = []
    for model in datasets:
        this_dataset = model.drop(columns=columns_to_drop)
        this_dataset = pd.concat([this_dataset, random_data_df], axis=1)
        datasets_no_skewed_scales.append(this_dataset)
    
    print(f"Train dataset")

    for model in datasets_no_skewed_scales:
        covariance_matrix_df, precision_matrix_df, support_matrix_df, proportion_matrix_df, estimator  = learn_graph_structure_adaptive_average_sklearn(
            model, 
            penalization='random',
            n_trials=n_trials,
            score_metric=score_metric,
            cv=cv,
            lam=None,
            alphas=alphas,
            threshold=threshold,
        )
        precision_matrixes.append(precision_matrix_df)

In [ ]:
np.save('data/power_tests/precision_matrices_normal_34.npy', np.array(precision_matrixes))